In [2]:
import json
import copy
import cobra
import cobrakbase

cobrakbase 0.2.7


In [10]:
kbase = cobrakbase.KBaseAPI()

In [7]:
map_json = None
with open('iMM904.Central carbon metabolism.json', 'r') as fh:
    map_json = json.loads(fh.read())

In [94]:
kbmodel = kbase.get_from_ws('iMM904_KBase3_ExtendedBiomassV1', 28277)

In [82]:
ws = 28277
fbamodel_json = kbase.get_object('iMM904_KBase3_ExtendedBiomass', ws)
print(fbamodel_json['id'])
print(fbamodel_json['name'])

iMM904
iMM904


In [99]:
cobra.core.model.configuration

Attribute,Description,Value
solver,Mathematical optimization solver,cplex
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,11


In [93]:
info.id

'iMM904_KBase3_ExtendedBiomassV1'

In [91]:
from cobrakbase.kbase_object_info import KBaseObjectInfo
info = kbase.get_object_info(70, ws)

In [88]:
kbmodel = kbase_api.get_from_ws('iMM904_KBase3_ExtendedBiomass', 28277)

In [95]:
cobra.io.save_json_model(kbmodel, '/Users/fliu/workspace/jupyter/iMM904_KBase3_ExtendedBiomassV1.json')

In [85]:
from cobrakbase.core.kbasefba.fbamodel_builder import FBAModelBuilder
from cobrakbase.core.kbasefba import FBAModel
from cobrakbase.core.kbaseobject import KBaseObject
from cobra.core import Model
b = FBAModelBuilder.from_kbase_json(fbamodel_json, info)
model = b.build()
print(type(model))
print(isinstance(model, FBAModel))
print(isinstance(model, Model))
print(isinstance(model, KBaseObject))
print(FBAModel.mro())
model

<class 'cobrakbase.core.kbasefba.fbamodel.FBAModel'>
True
True
True
[<class 'cobrakbase.core.kbasefba.fbamodel.FBAModel'>, <class 'cobrakbase.core.kbaseobject.KBaseObject'>, <class 'cobra.core.model.Model'>, <class 'cobra.core.object.Object'>, <class 'object'>]


Name,iMM904
Memory address,0x07fcbd84ce850
Number of metabolites,1226
Number of reactions,1578
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"e0, c0, m0, v0, x0, n0, r0, g0"


In [8]:
def adapt_map_to_kbase(refactored_escher_data2, map_id):
    refactored_escher_data = copy.deepcopy(refactored_escher_data2)
    refactored_escher_data[0]['map_name'] = map_id
    if not 'authors' in refactored_escher_data[0]:
        refactored_escher_data[0]['authors'] = []
    
    for node_uid in refactored_escher_data[1]['nodes']:
        node = refactored_escher_data[1]['nodes'][node_uid]
        #if node['node_type'] == 'metabolite':
        #    node['bigg_id'] = node['bigg_id'][:-3]
    
    for rxn_uid in refactored_escher_data[1]['reactions']:
        rxn_node = refactored_escher_data[1]['reactions'][rxn_uid]
        rxn_node['reversibility'] = 1 if rxn_node['reversibility'] else 0
        rxn_node['genes'] = []
        #rxn_node['bigg_id'] = rxn_node['bigg_id'][:-3]
        #rxn_node['metabolites'] = list(map(lambda x: {
        #    'coefficient': x['coefficient'], 
        #    'bigg_id': x['bigg_id'][:-3]}, rxn_node['metabolites']))
        for seg_uid in rxn_node['segments']:
            seg = rxn_node['segments'][seg_uid]
            if seg['b1'] == None:
                del seg['b1']
            if seg['b2'] == None:
                del seg['b2']
                
    for node_uid in refactored_escher_data[1]['nodes']:
        node = refactored_escher_data[1]['nodes'][node_uid]
        if 'node_is_primary' in node:
            node['node_is_primary'] = 1 if node['node_is_primary'] else 0
            
    kbase_escher = {
        "metadata" : refactored_escher_data[0],
        "layout" : refactored_escher_data[1]
    }
    
    return kbase_escher, rxn_node['metabolites']
kbase_json, a = adapt_map_to_kbase(map_json, 'iMM904.Central carbon metabolism')

In [21]:
for uid in kbase_json['layout']['reactions']:
    rnode = kbase_json['layout']['reactions'][uid]
    if rnode['bigg_id'].endswith('_c'):
        rnode['bigg_id'] = rnode['bigg_id'][:-2] + '_c0'
    if rnode['bigg_id'].endswith('_e'):
        rnode['bigg_id'] = rnode['bigg_id'][:-2] + '_e0'
    if rnode['bigg_id'].endswith('_m'):
        rnode['bigg_id'] = rnode['bigg_id'][:-2] + '_m0'
    if rnode['bigg_id'].endswith('_g'):
        rnode['bigg_id'] = rnode['bigg_id'][:-2] + '_g0'

In [22]:
kbase.save_object("iMM904.Central_carbon_metabolism","filipeliu:narrative_1582257065401", "KBaseFBA.EscherMap", kbase_json)

[[25,
  'iMM904.Central_carbon_metabolism',
  'KBaseFBA.EscherMap-1.0',
  '2021-03-23T17:16:52+0000',
  2,
  'filipeliu',
  56171,
  'filipeliu:narrative_1582257065401',
  'b624309c991a4b226065cbb8e479a26b',
  814910,
  {}]]